In [ ]:
  def collabarative_filtering(self):
    
        print('\n\nStarted building Collabarative Memory based Approach')

        books_data = self.books1.groupby('Book-Title')['Book-Rating'].count().reset_index().sort_values('Book-Rating', ascending=False).reset_index(drop=True)

        ## Using KNN
        ## Considering only books having atleast 50 ratings 
        books_data = books_data[books_data['Book-Rating']>50]

        ## Next we merge the above dataset with the books1 dataset 
        self.books_data_final = pd.merge(books_data,self.books1, left_on = 'Book-Title', right_on='Book-Title', how='left')

        #### Building a matrix with users as columns and book title as rows 
        self.books_user = self.books_data_final.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating_y').fillna(0)
        books_user_matrix = csr_matrix(self.books_user)

        ## Now we build with knn neighbours using cosine similarity 
        self.model = NearestNeighbors(metric = 'cosine', algorithm = 'brute', p=2)
        self.model.fit(books_user_matrix)



        ## Item Based Filtering 

        ## Taking Top rating books
        temp_data = pd.DataFrame(self.books1['Book-Title'].value_counts()).reset_index()
        temp_data.columns = ['Book-Title','Total-Rating']

        ## Merging with book1 dataframe and above data frame
        self.coll_data =  self.books1.merge(temp_data,left_on='Book-Title', right_on='Book-Title',how='left')
        self.coll_data.drop(['Year-Of-Publication','Publisher','Age','city','state','country'], axis=1, inplace=True)

        #### Taking top movies with atleast 80 user ratings
        self.coll_data = self.coll_data[self.coll_data['Total-Rating'] > 80].sort_values('Total-Rating',ascending=False).reset_index()   

        ## Creating the pivot table 
        self.ratings_matrix = self.coll_data.pivot_table(index='User-ID', columns = 'Book-Title', values='Book-Rating').fillna(0)


        print('\n\nDone building Collabarative Memory based Approach')

    
    def collabarative_knn_books(self, bookName):


        temp_data = self.books_data_final[self.books_data_final['Book-Title'] == bookName]
        if len(temp_data) == 0:
            return ["No book found"]

        ## Here I am getting 6 nearest neighbours(includes the entered book as well) for particular book
        distances, indices = self.model.kneighbors(self.books_user.loc[bookName].values.reshape(1, -1), n_neighbors = 6)

        ## Printing all recommended books
        similar_item = []
        print('\n\n\n')
        for i in range(0, len(distances.flatten())):
            if i == 0:
                print('Recommendations for book:',bookName,'\n\n')
            if i > 0:
                print(i,': ',self.books_user.index[indices.flatten()[i]],', with distance of',distances[0][i]) 
                similar_item.append(self.books_user.index[indices.flatten()[i]])
        print('\n\n\n')
        return similar_item

In [ ]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import plotly.io as pio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
books = pd.read_csv('/content/drive/MyDrive/Data/BookRecommendation/BX-Books.csv',
                        sep=';',
                        error_bad_lines=False,
                        encoding='latin-1')

books.head()


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version.



b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
users = pd.read_csv('/content/drive/MyDrive/Data/BookRecommendation/BX-Users.csv',
                        sep=';',
                        error_bad_lines=False,
                        encoding='latin-1')

users.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version.





,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/Data/BookRecommendation/BX-Book-Ratings.csv',
                        sep=';',
                        error_bad_lines=False,
                        encoding='latin-1')

ratings.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version.





,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
print(books.columns)
print(users.columns)
print(ratings.columns)

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Index(['User-ID', 'Location', 'Age'], dtype='object')
Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


In [ ]:
print(books.info())
print(users.info())
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), obje

In [ ]:
print(ratings.isna().sum())
print(books.isna().sum())
print(users.isna().sum())

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64
ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64
User-ID          0
Location         0
Age         110762
dtype: int64


In [ ]:
books = books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [ ]:
books[books['Publisher'].isnull()]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN


In [ ]:
books[books['Book-Author'].isnull()]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing


In [ ]:
#Dropping the Nan values as the percertage of null values is minimum

books.dropna(inplace=True)

In [ ]:
#Checking the column year-of-publication 

books['Year-Of-Publication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

In [ ]:
#Checking the characters values in Year of publication

books[books['Year-Of-Publication'].isin(['DK Publishing Inc','Gallimard'])]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


In [ ]:
#Here the values are misplaced. We will try to fix this 

books.loc[209538 ,'Publisher'] = 'DK Publishing Inc'
books.loc[209538 ,'Year-Of-Publication'] = 2000
books.loc[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'
books.loc[209538 ,'Book-Author'] = 'Michael Teitelbaum'

books.loc[221678 ,'Publisher'] = 'DK Publishing Inc'
books.loc[221678 ,'Year-Of-Publication'] = 2000
books.loc[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
books.loc[209538 ,'Book-Author'] = 'James Buckley'

books.loc[220731 ,'Publisher'] = 'Gallimard'
books.loc[220731 ,'Year-Of-Publication'] = 2003
books.loc[209538 ,'Book-Title'] = 'Peuple du ciel - Suivi de Les bergers '
books.loc[209538 ,'Book-Author'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'

In [ ]:
#Then converting into int datatype

books['Year-Of-Publication'] = books['Year-Of-Publication'].astype('int')

In [ ]:
#ratings = ratings.merge(books, on= "ISBN")[['User-ID', 'ISBN', 'Book-Rating']]
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
combine_book_rating = pd.merge(ratings, books, on='ISBN')
ratings  = combine_book_rating[['User-ID','Book-Title', 'Book-Rating']]
#combine_book_rating = ratings

In [ ]:
ratings

,User-ID,Book-Title,Book-Rating
0,276725,Flesh Tones: A Novel,0
1,2313,Flesh Tones: A Novel,5
2,6543,Flesh Tones: A Novel,0
3,8680,Flesh Tones: A Novel,5
4,10314,Flesh Tones: A Novel,9
...,...,...,...
1031131,276688,Mostly Harmless,0
1031132,276688,Gray Matter,7
1031133,276690,Triplet Trouble and the Class Trip (Triplet Tr...,0
1031134,276704,A Desert of Pure Feeling (Vintage Contemporaries),0


In [ ]:
combine_book_rating = combine_book_rating.dropna(axis=0, subset= ['Book-Title'])

In [ ]:
book_ratingCount = (combine_book_rating.groupby(by = ['Book-Title'])['Book-Rating'].count().reset_index().rename(columns = {'Book-Rating' : 'totalRatingCount'})[['Book-Title', 'totalRatingCount']])
book_ratingCount.head()

,Book-Title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [ ]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'Book-Title', right_on = 'Book-Title', how ='left')
rating_with_totalRatingCount.tail()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,totalRatingCount
1031128,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,45
1031129,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,21
1031130,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,1
1031131,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,1
1031132,276704,0806917695,5,Perplexing Lateral Thinking Puzzles: Scholasti...,Paul Sloane,1997,Sterling Publishing,1


In [ ]:
!pip install --quiet surprise

In [ ]:
from surprise import SVD, accuracy
from surprise.prediction_algorithms import matrix_factorization as mf
#surprise.prediction_algorithms.matrix_factorization.SVD
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from surprise.model_selection import train_test_split
import surprise



In [ ]:
ratings_1 = ratings[ratings['Book-Rating'] > 1]
Eligible = ratings_1['User-ID'].value_counts().rename_axis('User-ID').reset_index(name='counts')

In [ ]:
new = ratings_1.merge(Eligible[Eligible['counts'] >= 20], left_on = 'User-ID', right_on = 'User-ID', how ='inner')[['User-ID', 'Book-Title', 'Book-Rating']]

In [ ]:
new.head()

,User-ID,Book-Title,Book-Rating
0,2313,Flesh Tones: A Novel,5
1,2313,Ender's Game (Ender Wiggins Saga (Paperback)),9
2,2313,In Cold Blood (Vintage International),8
3,2313,Divine Secrets of the Ya-Ya Sisterhood : A Novel,9
4,2313,The Mistress of Spices,5


In [ ]:
Eligible = new['Book-Title'].value_counts().rename_axis('Book-Title').reset_index(name='counts')
Eligible[Eligible['counts'] >= 10]
new = new.merge(Eligible[Eligible['counts'] >= 10], left_on = 'Book-Title', right_on = 'Book-Title', how ='inner')[['User-ID', 'Book-Title', 'Book-Rating']]
new.shape

(62166, 3)

In [ ]:
#Create a Reader object with rating scale to pass to the algo
reader=Reader(rating_scale=(1,10))
data=surprise.dataset.Dataset.load_from_df(new,reader)



In [ ]:
#Split data into training and test datasets
#trainset, testset = train_test_split(data, test_size=0.15)
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [ ]:
#Choosing surprise SVD Algo
algo = surprise.prediction_algorithms.matrix_factorization.SVD(n_factors=100,n_epochs=9,lr_all=0.01,reg_all=0.1)

In [ ]:
#Train the trainset
algo.fit(trainset)
predictions = algo.test(testset)

In [ ]:
#Calculate Prediction accuracy of SVD
accuracy.rmse(predictions)


RMSE: 0.6987


0.6987454966354412

In [ ]:
new.head(n=100)

,User-ID,Book-Title,Book-Rating
0,2313,Flesh Tones: A Novel,5
1,10314,Flesh Tones: A Novel,9
2,77480,Flesh Tones: A Novel,8
3,98391,Flesh Tones: A Novel,9
4,115435,Flesh Tones: A Novel,5
...,...,...,...
95,78448,Divine Secrets of the Ya-Ya Sisterhood : A Novel,8
96,86189,Divine Secrets of the Ya-Ya Sisterhood : A Novel,5
97,222035,Divine Secrets of the Ya-Ya Sisterhood : A Novel,5
98,29526,Divine Secrets of the Ya-Ya Sisterhood : A Novel,10


In [ ]:
uid = 276847  # raw user id (as in the ratings file). They are **strings**!
iid = "3426029553"  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
#print(ratings[ (ratings['User-ID'] == uid) & (ratings['ISBN'] == iid) ])
print(new[ (new ['User-ID'] == uid) & (new['ISBN'] == iid)]) 
pred = algo.predict(uid, iid, r_ui=4, verbose=True)


   User-ID        ISBN  Book-Rating
0   276847  3426029553            8
user: 276847     item: 3426029553 r_ui = 4.00   est = 8.06   {'was_impossible': False}


In [ ]:
Eligible.value_counts()

ISBN        counts
000649840X  16        1
0553268449  18        1
0553213695  11        1
0553234811  14        1
0553250256  11        1
                     ..
0385722192  18        1
0385722206  69        1
0385722435  25        1
0385729332  23        1
3518368540  10        1
Length: 2609, dtype: int64

In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    
    # First map the predictions to each user
    top_n = defaultdict(list)
    top_n2 = defaultdict(list)
    
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        top_n2[uid] = list(zip(*top_n[uid]))[0]

        
    return top_n, top_n2

In [ ]:
# prediciton on teset and getting top predictions

top_n,top_n2 = get_top_n(predictions)

In [ ]:
top_n2

defaultdict(list,
            {2313: ("My Sister's Keeper : A Novel (Picoult, Jodi)",
              'Weirdos From Another Planet!',
              'Harry Potter and the Chamber of Secrets Postcard Book',
              'Dilbert: A Book of Postcards',
              "Scientific Progress Goes 'Boink':  A Calvin and Hobbes Collection",
              'Harry Potter and the Prisoner of Azkaban (Book 3)',
              "Harry Potter and the Sorcerer's Stone (Book 1)",
              'The Calvin &amp; Hobbes Lazy Sunday Book',
              'A Tree Grows in Brooklyn',
              'Griffin &amp; Sabine: An Extraordinary Correspondence'),
             10314: ("Scientific Progress Goes 'Boink':  A Calvin and Hobbes Collection",
              'Harry Potter and the Chamber of Secrets Postcard Book',
              'Dilbert: A Book of Postcards',
              'The Giving Tree',
              '84 Charing Cross Road',
              'Where the Red Fern Grows',
              "It's A Magical World: A Calvi

In [ ]:
pred_df = pd.DataFrame(top_n2).T
pred_df

,0,1,2,3,4,5,6,7,8,9
2313,"My Sister's Keeper : A Novel (Picoult, Jodi)",Weirdos From Another Planet!,Harry Potter and the Chamber of Secrets Postca...,Dilbert: A Book of Postcards,Scientific Progress Goes 'Boink': A Calvin an...,Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Sorcerer's Stone (Book 1),The Calvin &amp; Hobbes Lazy Sunday Book,A Tree Grows in Brooklyn,Griffin &amp; Sabine: An Extraordinary Corresp...
10314,Scientific Progress Goes 'Boink': A Calvin an...,Harry Potter and the Chamber of Secrets Postca...,Dilbert: A Book of Postcards,The Giving Tree,84 Charing Cross Road,Where the Red Fern Grows,It's A Magical World: A Calvin and Hobbes Coll...,The Neverending Story,The Secret Garden,Weirdos From Another Planet!
77480,"My Sister's Keeper : A Novel (Picoult, Jodi)",Harry Potter and the Chamber of Secrets Postca...,Dilbert: A Book of Postcards,Scientific Progress Goes 'Boink': A Calvin an...,Weirdos From Another Planet!,Calvin and Hobbes,East of Eden (Oprah's Book Club),Seabiscuit: An American Legend,Where the Red Fern Grows,The Authoritative Calvin and Hobbes (Calvin an...
98391,Scientific Progress Goes 'Boink': A Calvin an...,Harry Potter and the Chamber of Secrets Postca...,Weirdos From Another Planet!,Dilbert: A Book of Postcards,Where the Red Fern Grows,The Little Prince,Griffin &amp; Sabine: An Extraordinary Corresp...,Calvin and Hobbes,The Authoritative Calvin and Hobbes (Calvin an...,84 Charing Cross Road
115435,Dilbert: A Book of Postcards,Harry Potter and the Chamber of Secrets Postca...,Scientific Progress Goes 'Boink': A Calvin an...,Weirdos From Another Planet!,"My Sister's Keeper : A Novel (Picoult, Jodi)",A Tree Grows in Brooklyn,Lonesome Dove,"The Return of the King (The Lord of the Rings,...",Calvin and Hobbes,The Giving Tree
...,...,...,...,...,...,...,...,...,...,...
156534,Harry Potter and the Chamber of Secrets Postca...,"My Sister's Keeper : A Novel (Picoult, Jodi)",Weirdos From Another Planet!,Scientific Progress Goes 'Boink': A Calvin an...,The Little Prince,A Tree Grows in Brooklyn,The Giving Tree,84 Charing Cross Road,"The Two Towers (The Lord of the Rings, Part 2)",Calvin and Hobbes
243206,"My Sister's Keeper : A Novel (Picoult, Jodi)",Scientific Progress Goes 'Boink': A Calvin an...,Dilbert: A Book of Postcards,Weirdos From Another Planet!,Calvin and Hobbes,The Giving Tree,Harry Potter and the Chamber of Secrets Postca...,"The Return of the King (The Lord of the Rings,...",The Secret Garden,Fox in Socks (I Can Read It All by Myself Begi...
189722,Dilbert: A Book of Postcards,Harry Potter and the Chamber of Secrets Postca...,"My Sister's Keeper : A Novel (Picoult, Jodi)",Seabiscuit: An American Legend,The Neverending Story,Weirdos From Another Planet!,"The Return of the King (The Lord of the Rings,...","The Two Towers (The Lord of the Rings, Part 2)",Sister of My Heart,The Giving Tree
35185,Harry Potter and the Chamber of Secrets Postca...,Dilbert: A Book of Postcards,Scientific Progress Goes 'Boink': A Calvin an...,84 Charing Cross Road,"My Sister's Keeper : A Novel (Picoult, Jodi)",The Giving Tree,Calvin and Hobbes,Ender's Game (Ender Wiggins Saga (Paperback)),Seabiscuit: An American Legend,Harry Potter and the Goblet of Fire (Book 4)
